In [ ]:
pip install pyglet==1.5

In [ ]:
from robotic_warehouse.warehouse import Warehouse, RewardType, Action
import time
from tqdm import tqdm

In [ ]:
"""
(x,y)=(shelf_columns,shelf_columns)
tiny: (1, 3)
small: (2, 3)
medium: (2, 5)
large: (3, 5)
"""
shelf_rows=1
shelf_columns=3

"""
n_agents = # of robots
"""
n_agents=2

"""
difficulty:
    easy = 2
    medium = 1
    hard = 0.5
"""
request_queue_size=int(n_agents*1)

"""
RewardType(Enum):
    GLOBAL = 0
    INDIVIDUAL = 1
    TWO_STAGE = 2
"""
reward_type= RewardType.INDIVIDUAL

msg_bits=3
max_inactivity_steps=None
max_steps=500
sensor_range=1
column_height=8

In [ ]:
env = Warehouse(shelf_columns, column_height, shelf_rows, n_agents, msg_bits, sensor_range, request_queue_size, max_inactivity_steps, max_steps, reward_type)
env.reset()
env.render()

for _ in tqdm(range(1000000)):
    #env.render()
    actions = env.action_space.sample()
    new_obs, rewards, dones, info, delivered=env.step(actions)
    if delivered:
        print([new_obs, rewards, dones, info, delivered])  